# Tipos de datos algebraicos

## Enumeraciones

In [2]:
data Color = Red | Green | Blue

In [2]:
color :: Color
color = Red -- o Green, o Blue

`Red`, `Green`, `Blue` son __constructores__

## Constructores con argumentos

In [2]:
data TreeInt = Nil | Node TreeInt Int TreeInt

Un árbol binario de `Int` es:
* un árbol vacío (constructor `Nil`)
* un nodo raíz (constructor `Node`) cuyos hijos izquierdo y derecho también son árboles (`TreeInt`) y guarda un `Int` como dato.

El constructor `Node` recibe tres argumentos:
* hijo izquierdo (`TreeInt`)
* valor (`Int`)
* hijo derecho (`TreeInt`)

Los constructores __son funciones__:

In [19]:
:t Node

Node :: forall a. Tree a -> a -> Tree a -> Tree a

In [6]:
t :: TreeInt -- No hace falta
t = Node (Node Nil 2 Nil) 1 (Node (Node Nil 4 Nil) 3 (Node Nil 5 Nil))

## _Type variables_

Podemos hacer tipos de datos "genéricos" o "polimórficos" con __variables de tipo__ (_type variables_):

In [1]:
data Tree a = Nil | Node (Tree a) a (Tree a)

`Tree` es un __tipo parametrizado__ por el parámetro `a`

`a`: __cualquier__ tipo de dato

In [14]:
t :: Tree Char
t = Node (Node Nil 'b' Nil) 'a' (Node (Node Nil 'd' Nil) 'c' (Node Nil 'e' Nil))

In [15]:
t :: Tree Int
t = Node (Node Nil 2 Nil) 1 (Node (Node Nil 4 Nil) 3 (Node Nil 5 Nil))

Pueden usarse varias variables de tipo:

In [16]:
data Either a b = Left a | Right b

* Si se usa `Left` espera un dato de tipo `a`
* Si se usa `Right` espera un dato de tipo `b`

In [18]:
intChar1 :: Either Int Char
intChar1 = Left 1

intChar2 :: Either Int Char
intChar2 = Right 'a'

## Pattern matching

Cuando se hace _pattern matching_ __se matchean los constructores__:

In [14]:
leftOrRight :: Either a b -> String
leftOrRight (Left _) = "Left"
leftOrRight (Right _) = "Right"

In [16]:
leftOrRight intChar1

"Left"

In [17]:
leftOrRight intChar2

"Right"

In [ ]:
length [] = 0
length (x:xs) = 1 + length xs

En realidad es:

In [ ]:
length [] = 0
length ((:) x xs) = 1 + length xs

`(:)` es un __constructor__:

In [ ]:
data [a] = [] | (:) a [a]

## _Typeclasses_

Como "interfaces" o "clases abstractas"

* `Eq`: comparar con `==` `/=`
* `Show`: representar como string
* `Read`: crear valor a partir de string
* `Ord`: comparar con `<`, `>`, ...
* `Num`: operaciones `+`, `-`, `*`
* `Functor`
* `Applicative`
* `Monad`

La clase `Eq` está definida como:

In [32]:
class Eq a where
    (==) :: a -> a -> Bool
    (/=) :: a -> a -> Bool -- sólo hace falta implementar uno de los dos

Point es una __instancia__ de `Eq`: dos `Point x y` son "iguales" si están a la misma distancia del `Point 0 0`.

In [8]:
data Point = Point Int Int

instance Eq Point where
     (Point x1 y1) == (Point x2 y2) = (x1^2 + y1^2) == (x2^2 + y2^2)

In [11]:
Point 1 0 == Point 2 1

False

In [10]:
Point 0 5 == Point 3 4

True

## _Deriving_

Algunas clases pueden ser derivadas automáticamente:

In [12]:
data Point = Point Int Int

p = Point 1 2

print p

<interactive>:1:1: error:
    • No instance for (Show Point) arising from a use of ‘print’
    • In the expression: print p
      In an equation for ‘it’: it = print p

`Point` no es una instancia de `Show`

In [13]:
data Point = Point Int Int deriving Show

p = Point 1 2

print p

Point 1 2

### Functor

In [1]:
class Functor f where
    fmap :: (a -> b) -> f a -> f b

* Cómo "mapear" de `f a` a `f b`
* `f`: un __tipo parametrizado con un sólo parámetro__
* `f` puede ser
    * `Tree`: `data Tree a = Nil | Node (Tree a) a (Tree a)`
    * `Either a`: `data Either a b = Left a | Right b` (__no__ `Either`, ni `Either a b`)
    * `[]`
* `f` __no__ puede ser
    * `TreeInt`
    * `Point`
    * `Int`, `Char`, etc.

In [61]:
data Tree a = Nil | Node (Tree a) a (Tree a) deriving Show

instance Functor Tree where
    fmap f Nil = Nil
    fmap f (Node left x right) = Node (fmap f left) (f x) (fmap f right)

In [62]:
t = Node (Node Nil 2 Nil) 1 (Node (Node Nil 4 Nil) 3 (Node Nil 5 Nil))
fmap (show . (^2)) t

Node (Node Nil "4" Nil) "1" (Node (Node Nil "16" Nil) "9" (Node Nil "25" Nil))

Pasamos de `Tree Int` a `Tree String`

## Monad

In [4]:
f :: Int -> Int
f x = x^2

g :: Int -> Int
g x = x + 5

In [70]:
f 5
g 10

25

15

Supongamos que queremos loguear cada vez que se llama a las funciones:

In [71]:
fLog :: Int -> (Int, String)
fLog x = (f x, "Al cuadrado")

gLog :: Int -> (Int, String)
gLog x = (g x,"Mas 5")

In [72]:
fLog 5
gLog 25

(25,"Al cuadrado")

(30,"Mas 5")

¿Qué pasa si queremos componer `f` con `g` __y__ loggear al mismo tiempo?

¡Los tipos de `fLog` y `gLog` no son compatibles!

In [73]:
fgLog = fLog . gLog

<interactive>:1:16: error:
    • Couldn't match type ‘(Int, String)’ with ‘Int’
      Expected type: Int -> Int
        Actual type: Int -> (Int, String)
    • In the second argument of ‘(.)’, namely ‘gLog’
      In the expression: fLog . gLog
      In an equation for ‘fgLog’: fgLog = fLog . gLog

In [74]:
fgLog = fLog . fst . gLog

In [75]:
fgLog 5

(100,"Al cuadrado")

¡Perdimos el log de `g`!

In [76]:
fLog :: (Int, [String]) -> (Int, [String])
fLog (x,s) = (f x, s ++ ["Al cuadrado"])

gLog :: (Int, [String]) -> (Int, [String])
gLog (x,s) = (g x, s ++ ["Mas 5"])

In [78]:
fLog . gLog $ (2,[])

(49,["Mas 5","Al cuadrado"])

In [11]:
type Log = [String]

fLog :: Int -> (Int, Log)
fLog x = (f x, ["Al cuadrado"])

gLog :: Int -> (Int, Log)
gLog x = (g x, ["Mas 5"])

#### ¿Cómo podemos combinar las dos operaciones y concatenar los logs?

Primero creamos una función `fmapW` que nos permite aplicar otra función a un `(Int, Log)`, sólo al `Int`, y dejando el `Log` intacto:

In [12]:
fmapW :: (Int -> a) -> (Int, Log) -> (a, Log)
fmapW f (x,s) = (f x,s)

In [13]:
fmapW f (2,["Dos"])

(4,["Dos"])

¿Y si mapeamos una función que devuelva otro `(Int, Log)`?

In [15]:
r = gLog 5

In [16]:
r

(10,["Mas 5"])

In [39]:
fmapW fLog r

((100,["Al cuadrado"]),["Mas 5"])

Queremos que quede `(100,["Mas 5","Al cuadrado"])`... 

Para esto, creamos la función `joinW`:

In [41]:
fmapW fLog r

((100,["Al cuadrado"]),["Mas 5"])

In [40]:
joinW :: ((Int, Log), Log) -> (Int, Log)
joinW ((x,s),ss) = (x, ss ++ s)

In [42]:
joinW . fmapW fLog $ r

(100,["Mas 5","Al cuadrado"])

Esta función se llama `bindW`:

In [43]:
bindW :: (Int, Log) -> (Int -> (Int, Log)) -> (Int, Log)
bindW (x,s) f = joinW . fmapW f $ (x,s)

In [45]:
r = (5,[]) `bindW` gLog
r

(10,["Mas 5"])

In [46]:
r `bindW` fLog

(100,["Mas 5","Al cuadrado"])

Para no tener que poner el valor inicial del Log "a mano", creamos la función `returnW`:

In [48]:
returnW :: Int -> (Int, Log)
returnW x = (x, [])

In [49]:
returnW 5 `bindW` gLog `bindW` fLog

(100,["Mas 5","Al cuadrado"])

Acabamos de construir el `Monad` __`Writer`__

In [ ]:
class Applicative m => Monad where
    return :: a -> m a 
    (>>=) :: m a -> (a -> m b) -> m b -- bind

In [5]:
import Control.Monad.Writer

type Log = [String]

fLog :: Int -> Writer Log Int
fLog x = writer(f x,["Al cuadrado"])

gLog :: Int -> Writer Log Int
gLog x = writer(g x, ["Mas 5"])

In [6]:
runWriter(return 5 >>= gLog >>= fLog)

Line 1: Monad law, left identity
Found:
return 5 >>= gLog
Why not:
gLog 5

(100,["Mas 5","Al cuadrado"])

In [7]:
runWriter(gLog 5 >>= fLog)

(100,["Mas 5","Al cuadrado"])

In [8]:
runWriter(gLog 5 >>= gLog >>= gLog >>= fLog >>= fLog >>= gLog >>= fLog)

(25601600025,["Mas 5","Mas 5","Mas 5","Al cuadrado","Al cuadrado","Mas 5","Al cuadrado"])

### Notación __`do`__

In [59]:
gYf :: Writer Log Int
gYf = do
    x <- gLog 5
    fLog x

In [60]:
runWriter gYf

(100,["Mas 5","Al cuadrado"])

En general, los `Monad` sirven para:
* Combinar cálculos, cómputos, de alguna forma determinada: _"programmable semicolons"_
    * Writer
    * State: Mantener implícitamente un estado
    * Reader: Pasar un mismo parámetro a varias funciones
    * List
    * Error
    * Maybe
    * IO
    * ...
* Escribir código más parecido a los lenguajes imperativos, con la notación __`do`__